# M2177.003100 Deep Learning <br>Assignment #5 Part 1: Implementing and Training a Deep Q-Network

Copyright (C) Data Science Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Hyungyu Lee, November 2019

In this notebook, you will implement one of famous reinforcement learning algorithm, Deep Q-Network (DQN) of DeepMind. <br>
The goal here is to understand a basic form of DQN [1, 2] and learn how to use OpenAI Gym toolkit [3].<br>
You need to follow the instructions to implement the given classes.

1. [Play](#play) ( 50 points )

**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

### Submitting your work:
<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.  
Once you have done **two parts of the assignment**, run the *CollectSubmission.sh* script with your **Team number** as input argument. <br>
This will produce a zipped file called *[Your team number].tar.gz*. Please submit this file on ETL. &nbsp;&nbsp; (Usage: ./*CollectSubmission.sh* &nbsp; Team_#)

### Some helpful references for assignment #4 :
- [1] Mnih, Volodymyr, et al. "Playing atari with deep reinforcement learning." arXiv preprint arXiv:1312.5602 (2013). [[pdf]](https://www.google.co.kr/url?sa=t&rct=j&q=&esrc=s&source=web&cd=3&cad=rja&uact=8&ved=0ahUKEwiI3aqPjavVAhXBkJQKHZsIDpgQFgg7MAI&url=https%3A%2F%2Fwww.cs.toronto.edu%2F~vmnih%2Fdocs%2Fdqn.pdf&usg=AFQjCNEd1AJoM72DeDpI_GBoPuv7NnVoFA)
- [2] Mnih, Volodymyr, et al. "Human-level control through deep reinforcement learning." Nature 518.7540 (2015): 529-533. [[pdf]](https://www.nature.com/nature/journal/v518/n7540/pdf/nature14236.pdf)
- [3] OpenAI GYM website [[link]](https://gym.openai.com/envs) and [[git]](https://github.com/openai/gym)

## 0. OpenAI Gym

OpenAI Gym is a toolkit to support diverse environments for developing reinforcement learning algorithms. You can use the toolkit with Python as well as TensorFlow. Installation guide of OpenAI Gym is offered by [this link](https://github.com/openai/gym#installation) or just type the command "pip install gym" (as well as "pip install gym[atari]" for Part2). 

After you set up OpenAI Gym, you can use APIs of the toolkit by inserting <font color=red>import gym</font> into your code. In this assignment, you must build one of famous reinforcement learning algorithms whose agent can run on OpenAI Gym environments. Please check how to use APIs such as funcions interacting with environments in the followings.

In [1]:
import tensorflow as tf
import cv2 
import gym
import numpy as np
import os
import argparse
import sys
import random

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/pil-kso/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pil-kso/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pil-kso/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pil-kso

In [2]:
# Make an environment instance of CartPole-v0.
env = gym.make('CartPole-v0')

# Before interacting with the environment and starting a new episode, you must reset the environment's state.
state = env.reset()

#rendering game screens, do not need for assignment evaluation
# env.render() 

# You can check action space and state (observation) space.
num_actions = env.action_space.n
state_shape = env.observation_space.shape
print(num_actions)
print(state_shape)

# "step" function performs agent's actions given current state of the environment and returns several values.
# Input: action (numerical data)
#        - env.action_space.sample(): select a random action among possible actions.
# Output: next_state (numerical data, next state of the environment after performing given action)
#         reward (numerical data, reward of given action given current state)
#         terminal (boolean data, True means the agent is done in the environment)
next_state, reward, terminal, info = env.step(env.action_space.sample())

[2019-11-26 02:21:49,210] Making new env: CartPole-v0


2
(4,)


/home/pil-kso/anaconda3/envs/deep-learning-19/lib/python3.6/site-packages/gym/envs/registration.py:17: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


## 1. Implement a DQN agent
## 1) Overview of implementation in the notebook

The assignment is based on a method named by Deep Q-Network (DQN) [1,2]. You could find the details of DQN in the papers. The followings show briefly architecture of DQN and its training computation flow.

- (Pink flow) Play an episode and save transition records of the episode into a replay memory.
- (Green flow) Train DQN so that a loss function in the figure is minimized. The loss function is computed using main Q-network and Target Q-network. Target Q-network needs to be periodically updated by copying the main Q-network.
- (Purple flow) Gradient can be autonomously computed by tensorflow engine, if you build a proper optimizer.

![](image/architecture.png)

There are major 4 components, each of which needs to be implemented in this notebook. The Agent class must have an instance(s) of each class (Environment, DQN, ReplayMemory).
- Environment
- DQN 
- ReplayMemory
- Agent

![](image/components.png)



## 2) Design classes

In the code cells, there are only names of functions which are used in TA's implementation and their brief explanations. <font color='green'>...</font> means that the functions need more arguments and <font color='green'>pass</font> means that you need to write more codes. The functions may be helpful when you do not know how to start the assignment. Of course, you could change the functions such as deleting/adding functions or extending/reducing roles of the classes, <font color='red'> just keeping the existence of the classes</font>.

### Environment class

In [3]:
class Environment(object):
    def __init__(self, args):
        self.env = gym.make(args.env_name)
        self.display = args.display
        self.reward = 0
        self.terminal = True
        self.state = None
        self.state_shape = args.input_size


    def random_action(self):
        # Return a random action.
        return self.env.action_space.sample()
    
    def render_worker(self):
        # If display in your option is true, do rendering. Otherwise, do not.
        # you do not need to render in this assignment
        if self.display:
            self.env.render()
    
    def new_episode(self):
        # Start a new episode and return the first state of the new episode.
        state = self.env.reset()
        return state
    
    def step(self, action):
        self.state, self.reward, self.terminal, _ = self.env.step(action)

    def act(self, action):
        # Perform an action which is given by input argument and return the results of acting.
        self.step(action)
        self.render_worker()
        return self.state, self.reward, self.terminal

### ReplayMemory class

In [4]:
class ReplayMemory(object):
    def __init__(self, args):
        self.memory_size = args.memory_size
        self.batch_size = args.batch_size
        self.state_shape = args.input_size

        self.actions = np.empty(self.memory_size, dtype = np.uint8)
        self.rewards = np.empty(self.memory_size, dtype = np.integer)
        self.states = np.empty((self.memory_size, self.state_shape), dtype = np.float16)
        self.terminals = np.empty(self.memory_size, dtype = np.bool)
        self.count = 0
        self.current = 0

        self.prestates = np.empty((self.batch_size, self.state_shape), dtype = np.float16)
        self.poststates = np.empty((self.batch_size,self.state_shape), dtype = np.float16)
    
    def add(self, state, reward, action, terminal):
        assert state.shape[0] == self.state_shape

        self.actions[self.current] = action
        self.rewards[self.current] = reward
        self.states[self.current, :] = state[0]
        self.terminals[self.current] = terminal
        self.count = max(self.count, self.current + 1)
        self.current = (self.current + 1) % self.memory_size
        
    def get_state(self, index):
        assert self.count > 0
        index = index % self.count
        return self.states[index, :]

    def mini_batch(self):
        # Return a mini_batch from replay_memory according to your sampling method. (such as uniform-random sampling in DQN papers)
        indexes = []
        while len(indexes) < self.batch_size:
            while True:
                index = random.randint(1, self.count - 1)
                if index >= self.current and index - 1 < self.current:
                    continue
                if self.terminals[(index - 1):index].any():
                    continue
                break

            self.prestates[len(indexes), ...] = self.get_state(index - 1)
            self.poststates[len(indexes), ...] = self.get_state(index)
            indexes.append(index)

        actions = self.actions[indexes]
        rewards = self.rewards[indexes]
        terminals = self.terminals[indexes]

        return self.prestates, actions, rewards, self.poststates, terminals
        

### DQN class

In [5]:
class DQN(object):
    def __init__(self, args, sess, name="main"):
        self.session = sess 
        self.args = args
        self.input_size = args.input_size 
        self.output_size = args.output_size 
        self.net_name = name 
        self.build_network(name)
    
    def build_network(self, name, width = 10, L_rate = 1e-1):
        # Make your a deep neural network
        self.x=tf.placeholder(dtype=tf.float32, shape=[None, self.input_size])
        with tf.variable_scope(name):
            W1 = tf.get_variable('W1',shape=[self.input_size, width],initializer=tf.contrib.layers.xavier_initializer()) 
            W2 = tf.get_variable('W2',shape=[width, self.output_size],initializer=tf.contrib.layers.xavier_initializer())

            L1=tf.nn.tanh(tf.matmul(self.x,W1))

            self.Q_pre = tf.matmul(L1,W2) 
            self.y=tf.placeholder(dtype=tf.float32, shape=[None, self.output_size]) 

            self.loss = tf.reduce_sum(tf.square(self.y - self.Q_pre)) 
            self.build_optimizer()
    
    def build_optimizer(self):
        # Make your optimizer 
        self.train = tf.train.AdamOptimizer(learning_rate=self.args.learning_rate).minimize(self.loss)
    
    def train_network(self, replay_memory):
        # Train the prediction_Q network using a mini-batch sampled from the replay memory
        for index in range(self.args.replay_num): 
            s_r, a_r, r_r, s1_r, d_r = replay_memory.mini_batch()
            Q = self.predict(s_r) 

            if d_r.any(): 
                Q[0, a_r] = -100 
            else: 
                Q[0, a_r] = r_r + self.args.discount_factor * np.max(self.predict(s1_r)) 
            self.update_target_network(np.reshape(s_r, [-1, self.input_size]), Q)

    
    def update_target_network(self, x, y):
        self.session.run(self.train, feed_dict={self.x : x, self.y : y})
    
    def predict(self, state):
        s_t = np.reshape(state, [-1, self.input_size]) 
        return self.session.run(self.Q_pre, feed_dict={self.x : s_t})


### Agent class

In [6]:
class Agent(object):
    def __init__(self, args, sess, name = 'main'):
        self.args = args
        self.sess = sess

        self.env = Environment(self.args)
        self.memory = ReplayMemory(self.args)
        
        self.returns = []
        self.main_dqn = DQN(self.args, sess)
        sess.run(tf.global_variables_initializer())
        
        self.saver = tf.train.Saver()
    
    def select_action(self, state, epsilon):
        # Select an action according ε-greedy. You need to use a random-number generating function and add a library if necessary.
        if epsilon > np.random.rand(1): 
            action = self.env.random_action() 
        else: 
            action = np.argmax(self.main_dqn.predict(state))
        return action
    
    def train(self):
        # Train your agent 
        # Several hyper-parameters are determined by your choice
        # Keep epsilon-greedy action selection in your mind 
        self.returns = []
        
        for step in range(self.args.num_episode):
            s = self.env.new_episode()
            e = 1. / ((step/10)+1) 
            rall = 0 
            d = False 
            count=0
            
            while not d and count < 5000:
                count+=1
                a = self.select_action(s, e)
                s1, r, d = self.env.act(a)
                self.memory.add(s, r, a, d)
                rall += r 
                s = s1
            if step % 10 == 1 : 
                for _ in range(self.args.batch_size): 
                    self.main_dqn.train_network(self.memory)

            self.returns.append(rall) 
                    
    def play(self):
        # Test your agent 
        # When performing test, you can show the environment's screen by rendering if you want
        s = self.env.new_episode()
        rall = 0 
        d = False 
        count=0

        while not d and count < 5000:
            count+=1
            a = self.select_action(s, 0)
            s1, r, d = self.env.act(a)
            self.memory.add(s, r, a, d)
            rall += r 
            s = s1
        return rall
    
    def save(self):
        checkpoint_dir = 'cartpole'
        if not os.path.exists(checkpoint_dir):
            os.mkdir(checkpoint_dir)
        self.saver.save(self.sess, os.path.join(checkpoint_dir, 'trained_agent'))
        
    def load(self):
        checkpoint_dir = 'cartpole'
        self.saver.restore(self.sess, os.path.join(checkpoint_dir, 'trained_agent'))

## 2. Train your agent 

Now, you train an agent to play CartPole-v0. Options class is the collection of hyper-parameters that you can choice. Usage of Options class is not mandatory.<br>
The maximum value of total reward which can be aquired from one episode is 200. 
<font color='red'>**You should show learning status such as the number of observed states and mean/max/min of rewards frequently (for instance, every 100 states).**</font>

In [7]:
parser = argparse.ArgumentParser(description="CartPole")
parser.add_argument('--env-name', default='CartPole-v0', type=str,
                    help="Environment")
parser.add_argument('--replay-num', default=10, type=int, help="")
parser.add_argument('--batch-size', default=50, type=int, help="")
parser.add_argument('--memory-size', default=50000, type=int, help="")
parser.add_argument('--history-length', default=4, type=int, help="")
parser.add_argument('--learning-rate', default=0.1, type=float, help="")
parser.add_argument('--num-episode', default=2000, type=int, help="")
parser.add_argument('--discount-factor', default=0.9, type=float, help="")
parser.add_argument('--input-size', default=state_shape[0], help="")
parser.add_argument('--output-size', default=num_actions, help="")
parser.add_argument('--display', default=False, help="")


"""
You can add more arguments.
for example, visualize, memory_size, batch_size, discount_factor, eps_max, eps_min, learning_rate, train_interval, copy_interval and so on
"""

with tf.Session() as sess:
#     args = parser.parse_args()
    args = parser.parse_known_args()[0]
    myAgent = Agent(args, sess) # It depends on your class implementation
    myAgent.train()
    myAgent.save()

'\nYou can add more arguments.\nfor example, visualize, memory_size, batch_size, discount_factor, eps_max, eps_min, learning_rate, train_interval, copy_interval and so on\n'

## <a name="play"></a> 3. Test the trained agent ( 50 points )

Now, we test your agent and calculate an average reward of 20 episodes.
- 0 <= average reward < 50 : you can get 0 points
- 50 <= average reward < 100 : you can get 10 points
- 100 <= average reward < 190 : you can get 35 points
- 190 <= average reward <= 200 : you can get 50 points

In [8]:
config = tf.ConfigProto()
# If you use a GPU, uncomment
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config.log_device_placement = False
config.gpu_options.allow_growth = True
tf.reset_default_graph()
with tf.Session(config=config) as sess:
    args = parser.parse_known_args()[0] # You set the option of test phase
    myAgent = Agent(args, sess, 'test') # It depends on your class implementation
    myAgent.load()
    rewards = []
    for i in range(20):
        r = myAgent.play() # play() returns the reward cumulated in one episode
        rewards.append(r)
    mean = np.mean(rewards)
    print(rewards)
    print(mean)

[2019-11-26 02:22:05,095] Making new env: CartPole-v0


INFO:tensorflow:Restoring parameters from cartpole/trained_agent


[2019-11-26 02:22:08,853] Restoring parameters from cartpole/trained_agent


[200.0, 200.0, 198.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 199.0, 200.0, 200.0, 200.0, 200.0, 200.0]
199.85
